In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,313 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,

In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Reviews").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [6]:
# Recreate the vine_table
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8EWA1OFT84NX|          5|            0|          0|   N|                Y|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|                N|
| RL5ESX231LZ0B|          5|            0|          0|   N|                Y|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|                Y|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|                Y|
|R13EPSFP5DODN5|          4|            0|          0|   N|                Y|
| R6RBP4HTE67SY|          5|            0|          0|   N|                Y|
|R15B3EU40RSU2W|          5|            0|          0|   N|                Y|
| RP4DD53A4ZJA2|          5|            0|          0|   N|                Y|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|     

In [7]:
# Filter hte data to retrieve all rows where total_votes count is equal to or greater than 20 
votes_df = vine_df.filter("total_votes>=20").show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
|R3UUR313K5VVTL|          1|            1|         24|   N|                Y|
| RODE8K12S7148|          5|           41|         45|   N|     

In [8]:
# Filter new df to retrieve all rows where number of helpful_votes divided by total_votes is equal to or greater than 50%
# Create the vine_table. DataFrame
helpful_votes = vine_df.filter((vine_df.helpful_votes / vine_df.total_votes) >= 0.5)
helpful_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2C99DJEO4RZ4K|          5|            3|          4|   N|                Y|
|R2DRL5NRODVQ3Z|          5|            2|          2|   N|                Y|
|R3T9B92MDDHKMM|          2|            5|          5|   N|                Y|
|R1OK6O3H5N02VG|          5|           11|         12|   N|                Y|
|R13S4RY124ZQB3|          3|            1|          1|   N|                Y|
|R1S70EUSH1Y1I9|          1|            1|          2|   N|                Y|
| RIQEOQGRAJBQJ|          5|            1|          1|   N|                Y|
|R3UNJA28ABKM3T|          5|            1|          1|   N|                Y|
| RXGYKJONSLT54|          5|            1|          1|   N|                Y|
| RUW8OQE13AUZG|          5|            2|          3|   N|     

In [15]:
# Retrieve all rows where a review was written as part of the Vine program. 

paid_vine = helpful_votes.filter(helpful_votes['vine'] == 'Y')
paid_vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DQZOJNMI37KI|          4|            1|          2|   Y|                N|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R2GTGOL5UM0TYI|          4|            4|          4|   Y|                N|
|R1A88KV72B9ZFZ|          5|            1|          2|   Y|                N|
|R2E9V8X9RJOVNR|          1|            2|          3|   Y|                N|
| R86G1WYZGPF8G|          4|            1|          1|   Y|                N|
|R1D56IAOJYT1YN|          5|            2|          3|   Y|                N|
|R1ZCWONQO0HCIE|          5|            1|          2|   Y|                N|
| RI4TRFMPG57A4|          4|            1|          2|   Y|                N|
|R1R66PK8E7OUF2|          4|            3|          6|   Y|     

In [18]:
# Retrieve all rows where a review was written as part of the Vine program. 

unpaid_not_vine = helpful_votes.filter(helpful_votes['vine'] == 'N')
unpaid_not_vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2C99DJEO4RZ4K|          5|            3|          4|   N|                Y|
|R2DRL5NRODVQ3Z|          5|            2|          2|   N|                Y|
|R3T9B92MDDHKMM|          2|            5|          5|   N|                Y|
|R1OK6O3H5N02VG|          5|           11|         12|   N|                Y|
|R13S4RY124ZQB3|          3|            1|          1|   N|                Y|
|R1S70EUSH1Y1I9|          1|            1|          2|   N|                Y|
| RIQEOQGRAJBQJ|          5|            1|          1|   N|                Y|
|R3UNJA28ABKM3T|          5|            1|          1|   N|                Y|
| RXGYKJONSLT54|          5|            1|          1|   N|                Y|
| RUW8OQE13AUZG|          5|            2|          3|   N|     

In [34]:
# Total number of reviews for paid
from pyspark.sql.functions import count
total_paid = paid_vine.count()
total_paid

5028

In [35]:
# Total number of reviews for unpaid 
total_unpaid = unpaid_not_vine.count()
total_unpaid

495942

In [27]:
# 5 star reviews for paid 
five_star_paid = paid_vine.filter(paid_vine['star_rating'] == 5).count()
five_star_paid

2368

In [32]:
# 5 star reviews for unpaid
five_star_unpaid = unpaid_not_vine.filter(unpaid_not_vine['star_rating'] == 5).count()
five_star_unpaid

265843

In [37]:
# Percentage of 5 star reviews for paid 
percentage_paid = (five_star_paid / total_paid) * 100 
percentage_paid

47.09626093874304

In [38]:
# Percentage of 5 star reviews for unpaid 
percentage_unpaid = (five_star_unpaid / total_unpaid) * 100 
percentage_unpaid

53.603647200680726